In [11]:
from dotenv import load_dotenv
load_dotenv()

from typing import Annotated, List, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver

from chains import get_llm

In [2]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = get_llm(api_key="OLLAMA")  
generate = prompt | llm

In [3]:
essay = ""

request = HumanMessage(
    content="Write an essay explaining the central theme of the count of Monte Cristo"
)

for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content


Certainly! Here is a five-paragraph essay exploring the central theme of "The Count of Monte Cristo" by Alexandre Dumas:

### The Central Theme of "The Count of Monte Cristo"

"The Count of Monte Cristo," or "The Count of the Mont-Croix" as it was originally titled, is an epic tale that delves deeply into themes of revenge and redemption. At its core lies a compelling exploration of how individuals can transform their lives through perseverance and forgiveness in response to unjust treatment.

At first glance, Edmond Dantès's story appears to be one of vengeance against those who betrayed him. His imprisonment by the machinations of his treacherous political allies is the catalyst for his journey into the depths of the labyrinthine Monte Cristo Castle. However, as he navigates this castle and encounters various characters, including a mysterious man named Fernand, Dantès's perspective begins to shift. He realizes that his initial desire for vengeance has blinded him to the suffering he

In [5]:
chat = ChatPromptTemplate.from_messages([
   ("system","You are a helpful AI Assistant with a sense of humor"),
   ("human","Hi how are you?"),
   ("ai","I am good. How can I help you?"),
   ("human","{input}")
])
print(chat)

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI Assistant with a sense of humor'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hi how are you?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='I am good. How can I help you?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI Assistant with a sense of humor'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hi how are you?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='I am good. How can I help you?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]

In [6]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [7]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Your essay provides a comprehensive analysis of the central theme in "The Count of Monte Cristo," which is well-structured and insightful. Here are some detailed critiques and recommendations for enhancing your essay:

### Critique

1. **Depth and Breadth**:
   - The essay covers several aspects of the central theme, including revenge, redemption, forgiveness, and human suffering. However, it could benefit from a more focused exploration of one or two key elements to ensure depth.
   
2. **Character Development**:
   - While you mention Fernand's character effectively, there is potential for deeper analysis. How does his journey reflect the theme of vengeance versus redemption? What specific actions demonstrate this?
   
3. **Symbolism and Setting**:
   - The setting of Monte Cristo Castle plays a crucial role in illustrating the themes. You could delve into how the castle symbolizes different stages of Edmond's transformation, such as isolation, enlightenment, and eventual release.
  

In [8]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Here is the revised essay incorporating the critiques and recommendations:

---

### The Central Theme of "The Count of Monte Cristo"

"The Count of Monte Cristo," by Alexandre Dumas, delves deeply into the themes of revenge and redemption. At its core lies Edmond Dantès's transformative journey from a vengeful prisoner to a benefactor who embodies the power of forgiveness.

#### The Role of Revenge

Edmond Dantès's initial desire for vengeance against those who betrayed him serves as the catalyst for his epic quest. His imprisonment by political allies like Fernand and Abbé Faria sets in motion a series of events that lead to his transformation. However, through his interactions with various characters, particularly Fernand and Abbé Faria, Edmond begins to see the error of his ways.

Fernand's character exemplifies the theme of revenge. Initially seeking vengeance against those who wronged him, including Edmond, Fernand becomes more successful and wealthy but ultimately seeks reconcil

In [12]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


async def generation_node(state: State) -> State:
    return {"messages": [await generate.ainvoke(state["messages"])]}


async def reflection_node(state: State) -> State:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]
    res = await reflect.ainvoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}


builder = StateGraph(State)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_edge(START, "generate")


def should_continue(state: State):
    if len(state["messages"]) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
memory = InMemorySaver()
graph = builder.compile(checkpointer=memory)

In [13]:
config = {"configurable": {"thread_id": "1"}}

In [ ]:
async for event in graph.astream(
    {
        "messages": [
            HumanMessage(
                content="Generate an essay on the topicality of The Little Prince and its message in modern life"
            )
        ],
    },
    config,
):
    print(event)
    print("---")

{'generate': {'messages': [AIMessage(content='### The Topicality of "The Little Prince" and Its Message in Modern Life\n\n"The Little Prince," written by Antoine de Saint-Exupéry, is a timeless tale that continues to resonate with readers across generations. Despite being published over seven decades ago, the story\'s themes remain relevant and topical in modern life. The novel offers profound insights into human nature, relationships, and our place within society, making it an invaluable text for contemporary audiences.\n\nOne of the most significant messages conveyed by "The Little Prince" is its emphasis on the importance of genuine connections over superficial ones. In a world often characterized by rapid communication through digital platforms, people can feel increasingly isolated despite being surrounded by others. The little prince\'s journey teaches us that true friendships and meaningful relationships are built upon empathy, understanding, and mutual respect – qualities that 